# CIFAR-10 Image Classification 

In [1]:
# Import Libraries

import numpy
import time
import calendar
from six.moves import cPickle as pickle
import numpy as np
import cv2
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score

##### General parameters

In [2]:
# General Parameters

imageSize = 32  
channels = 3    
classes = 10    
trainingDataSize = 50000    
testDataSize = 10000        
trainingDataFiles = ('./dataset/cifar-10-batches-py/data_batch_1', './dataset/cifar-10-batches-py/data_batch_2', './dataset/cifar-10-batches-py/data_batch_3', './dataset/cifar-10-batches-py/data_batch_4','./dataset/cifar-10-batches-py/data_batch_5') 
testDataFile = './dataset/cifar-10-batches-py/test_batch' 
pcaFileName = 'pca' 
svmFileName = 'svm' 

##### Load the data from the given filename

In [3]:
def loadData(filename):
    '''
    Load the data from the given filename

    Parameters:
    -----------
    filename: string
        The name of the file containing the data to load

    Returns:
    --------
    theSet['data']:     array of images
    theSet['labels']:   array of labels
    '''
    f = open(filename, 'rb')
    theSet = pickle.load(f,encoding='latin1')
    f.close()

    return theSet['data'], theSet['labels']

##### Convert the images from CIFAR-10 format, to an array with each image of 32 X 32 X 3 size

In [4]:
def convertImages(origImages):
    '''
    Convert the images from CIFAR-10 format, to an array of 10000 images each is 32 X 32 X 3 size

    Parameters:
    -----------
    origImages: array
        array of images in the CIFAR-10 format

    Returns:
    --------
    images:     array of images each in 32 X 32 X 3 size
    '''
    images = np.reshape(origImages,(-1, channels, imageSize, imageSize))
    images = np.transpose(images, (0,2,3,1))

    return images

##### Load all the training data from all files

In [5]:
def loadTrainingData(filenames):
    '''
    Load all the training data from all files

    Parameters:
    -----------
    filenames: array of string
        An array The name of the file containing the data to load

    Returns:
    --------
    trainingImages: array of the training set images
    trainingLabels: array of the training set labels
    '''

    #Pre-allocate the arrays
    trainingImages = np.zeros(shape=[trainingDataSize, imageSize, imageSize, channels], dtype=numpy.uint8)
    trainingLabels = np.zeros(shape=[trainingDataSize], dtype=int)

    start=0
    for fileName in filenames:

      origImages, labels = loadData(fileName)
      images = convertImages(origImages)

      numOfImages = len(images)
      end = start + numOfImages
      # print(numOfImages)
      trainingImages[start:end, :] = images
      trainingLabels[start:end] = labels
      start = end

    return trainingImages, trainingLabels

##### Load the test data

In [6]:
def loadTestData(filename):
    '''
    Load the test data

    Parameters:
    -----------
    filename: string
        The name of the file containing the test data

    Returns:
    --------
    testImages: array of images of the test data
    testLabels: array of labels of the test data
    '''

    origTestImages, testLabels = loadData(filename)
    testImages = convertImages(origTestImages)

    return testImages, testLabels

##### Returns the current time in seconds since epoch

In [7]:
def currentTime():
    '''
    Returns the current time in seconds since EPOC
    Used to measure how much time each phase took

    Returns:
    --------
    the current time in second since EPOC
    '''

    return calendar.timegm(time.gmtime())

##### Calculate the HOG descriptors of the given images

In [8]:
winSize = imageSize
blockSize = 12
blockStride = 4
cellSize = 4
nbins = 9
derivAperture = 1
winSigma = -1.
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = True
nlevels = 64
signedGradient = True
hog = cv2.HOGDescriptor((winSize,winSize),(blockSize, blockSize),(blockStride,blockStride),(cellSize,cellSize),nbins,derivAperture, winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels,signedGradient)

In [9]:
# HOG for all images in image set

def calcHOG(images):
  '''
    Calculate the HOG descriptors of the given images

    Parameters:
    -----------
    images: an array of images
        The images to which a HOG calculation should be applied

    Returns:
    --------
    hogDescriptors: an array of HOG vectors, 5832 components each
    '''
    
  hogDescriptors = []
  for image in images:
    hogDescriptors.append( hog.compute(image) )

  hogDescriptors = np.squeeze(hogDescriptors)
  return hogDescriptors

#### Algorithm Implementation

In [10]:
from google.colab import drive
drive.mount('/content/Drive', force_remount=True)

Mounted at /content/Drive


In [11]:
!mkdir -p "/content/dataset"
!tar -xzf "/content/Drive/MyDrive/CIFAR-10/cifar-10-python.tar.gz" -C "/content/dataset"

###### Load the train set

In [12]:
print("Loading the training set..."),
tik = currentTime()
trainingImages, trainingLabels = loadTrainingData(trainingDataFiles)
print("Took: " + str(currentTime()-tik) + " sec" )

Loading the training set...
Took: 1 sec


###### Load the test set

In [13]:
print("Loading the test set..."),
tik = currentTime()
testImages, testLabels = loadTestData(testDataFile)
print("Took: " + str(currentTime() - tik) + " sec")

Loading the test set...
Took: 0 sec


###### Final dataset

In [14]:
labels = np.concatenate((trainingLabels,testLabels))

In [15]:
features = np.concatenate((trainingImages,testImages))

###### Create a HOG descriptor from these images

In [16]:
print("Creating HOG descriptors from the image set..."),
tik = currentTime()
imageHogDescriptors = calcHOG(features)
print("Took: " + str(currentTime() - tik) + " sec")

Creating HOG descriptors from the image set...
Took: 3 sec


###### Reduce the dimension of the HOG descriptors using PCA

In [17]:
print("Reducing dimension of the HOG descriptors " + "..."),
tik = currentTime()
pca = PCA(n_components=0.99)
imageHogProjected = pca.fit_transform(imageHogDescriptors)
print("Took: " + str(currentTime() - tik) + " sec")

Reducing dimension of the HOG descriptors ...
Took: 61 sec


###### Save it as a pca file

In [18]:
print("Save it as a PCA file..."),
tik = currentTime()
pcaFile = open(pcaFileName, 'wb')
pickle.dump(pca, pcaFile)
pcaFile.close()
print("Took: " + str(currentTime() - tik) + " sec")

Save it as a PCA file...
Took: 0 sec


###### Train the SVM model using the reduced HOG descriptor

In [ ]:
svmm = svm.SVC()
svmm.fit(imageHogProjected,labels)

print("Save it as a SVM file..."),
tik = currentTime()
svmFile = open(svmFileName, 'wb')
pickle.dump(svmm, svmFile)
svmFile.close()
print("Took: " + str(currentTime() - tik) + " sec")